# Mortimer Jordan HS — Education Snapshot (ALSDE 2024)

This notebook builds outcome-focused charts for Mortimer Jordan High School from **ALSDE Supporting Data (2024)**:
- Graduation Rate, College & Career Readiness (CCR), Academic Growth, Chronic Absenteeism (from **Accountability**)
- Educator experience (from **Educator Credentials/Experienced**)

**Filters used in code**  
System = *Jefferson County* (037), School = *Mortimer Jordan High School* (0610), Group slices = *All*.

**Guardrail**  
If you load a statewide-only export (SystemCode=000/SchoolCode=0000), the notebook will alert you to re-export.

Last run: Aug 21, 2025.


In [ ]:
# Bootstrap: folders and paths (safe to re-run)
from pathlib import Path
import shutil, pandas as pd, matplotlib.pyplot as plt

ROOT = Path.cwd().resolve()
DATA_RAW = ROOT / "data" / "raw"
DATA_PROCESSED = ROOT / "data" / "processed"
CHARTS = ROOT / "charts"
OUTPUTS = ROOT / "outputs"

for p in [DATA_RAW, DATA_PROCESSED, CHARTS, OUTPUTS]:
    p.mkdir(parents=True, exist_ok=True)

# Helper to stamp chart footer
def add_footer(ax, text):
    ax.figure.text(0.01, 0.01, text, ha="left", va="bottom", fontsize=9, alpha=0.8)

plt.rcParams["figure.figsize"] = (12, 6)

In [ ]:

# Load ALSDE CSVs from data/raw; prefer explicit filenames, otherwise glob patterns
import pandas as pd

def find_one(patterns):
    for pat in patterns:
        hits = sorted(DATA_RAW.glob(pat))
        if hits:
            hits.sort(key=lambda p: p.stat().st_mtime, reverse=True)
            return hits[0]
    raise FileNotFoundError(f"No file found in {DATA_RAW} for patterns: {patterns}")

ACC_PATH  = find_one(["*Accountability*.csv", "*accountability*.csv"])
# CCR file optional; not required since Accountability contains Grad/CCR
try:
    CCR_PATH = find_one(["*CCR*Grad*.csv", "*CCR*Rate*.csv", "*CCR*.csv"])
except FileNotFoundError:
    CCR_PATH = None
# Educator Experience / Credentials
EXP_PATH  = find_one(["*Educator*Credential*.csv", "*Experienced*.csv", "*Educator*.csv"])

print("Using:")
print(" ACC:", ACC_PATH.name)
print(" CCR:", CCR_PATH.name if CCR_PATH else "(none)")
print(" EXP:", EXP_PATH.name)

accountability = pd.read_csv(ACC_PATH)
educator       = pd.read_csv(EXP_PATH)
ccr = pd.read_csv(CCR_PATH) if CCR_PATH else None


In [ ]:

import pandas as pd
import numpy as np

DISTRICT_NAME = "Jefferson County"
SCHOOL_NAME   = "Mortimer Jordan High School"
KEY_INDS = ["Academic Growth","Graduation Rate","College and Career Readiness","Chronic Absenteeism"]

def keep_all_groups(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for col in ["Grade","Gender","Race","Ethnicity","Sub Population"]:
        if col in out.columns:
            out = out[out[col].astype(str).str.contains("All", case=False, na=False)]
    return out

def first_col(df: pd.DataFrame, names):
    # exact then contains
    for n in names:
        for c in df.columns:
            if c.lower() == n.lower():
                return c
    for n in names:
        for c in df.columns:
            if n.lower() in c.lower():
                return c
    return None

def assert_not_statewide_only(df, label):
    sys_vals = set(df["System"].astype(str).str.strip().unique()) if "System" in df.columns else set()
    sch_vals = set(df["School"].astype(str).str.strip().unique()) if "School" in df.columns else set()
    if sys_vals == {"Alabama State Department of Education"} and sch_vals == {"Alabama State Department of Education"}:
        raise ValueError(f"{label} appears to be statewide-only (SystemCode=000, SchoolCode=0000). "
                         "Please export All Systems/Schools or Jefferson County / Mortimer Jordan HS.")

for df, label in [(accountability, "Accountability"), (educator, "Educator")]:
    assert_not_statewide_only(df, label)

acc_all = keep_all_groups(accountability)
acc_mj  = acc_all[(acc_all["System"] == DISTRICT_NAME) & (acc_all["School"] == SCHOOL_NAME)].copy()

if acc_mj.empty:
    raise ValueError("No Mortimer Jordan rows found in Accountability after filtering. "
                     "Verify CSVs and group slices (All).")

acc_kpi = (acc_mj[acc_mj["Indicator"].isin(KEY_INDS)]
           .groupby("Indicator", as_index=False)["Score"].max()
           .sort_values("Indicator"))
display(acc_kpi)

# Educator experience summary
edu_all = keep_all_groups(educator)
edu_mj  = edu_all[(edu_all["System"] == DISTRICT_NAME) & (edu_all["School"] == SCHOOL_NAME)].copy()

exp_pct_col = first_col(edu_mj, ["Experienced Rate","Experienced %","Percent Experienced"])
nov_pct_col = first_col(edu_mj, ["Inexperienced Rate","Novice Rate","Percent Inexperienced"])
exp_cnt_col = first_col(edu_mj, ["Experienced Count","Number Experienced"])
tot_cnt_col = first_col(edu_mj, ["Total Count","Total"])

rows = []
if exp_pct_col:
    rows.append({"Metric":"Experienced Educators (%)", "Value": pd.to_numeric(edu_mj[exp_pct_col], errors="coerce").max()})
elif exp_cnt_col and tot_cnt_col:
    num = pd.to_numeric(edu_mj[exp_cnt_col], errors="coerce").sum()
    den = pd.to_numeric(edu_mj[tot_cnt_col], errors="coerce").sum()
    rows.append({"Metric":"Experienced Educators (%)", "Value": (num/den*100) if den else np.nan})

if nov_pct_col:
    rows.append({"Metric":"Inexperienced/Novice (%)", "Value": pd.to_numeric(edu_mj[nov_pct_col], errors="coerce").max()})

exp_summary = pd.DataFrame(rows)
display(exp_summary)


In [ ]:

# Plot: Graduation & CCR
import matplotlib.pyplot as plt

grad_ccr = acc_kpi[acc_kpi["Indicator"].isin(["Graduation Rate","College and Career Readiness"])]
plt.figure(figsize=(10,5))
plt.bar(grad_ccr["Indicator"], grad_ccr["Score"])
plt.title("Mortimer Jordan HS — Graduation & CCR (ALSDE 2024)")
plt.ylabel("Percent")
plt.tight_layout()
add_footer(plt.gca(), "Source: ALSDE Supporting Data • 2024 • All groups (Jefferson County / MJHS)")
out1 = CHARTS / "mjhs_grad_ccr_2024.png"
plt.savefig(out1, dpi=200); plt.show()
print("Saved chart:", out1)

# Plot: Educator experience
if not exp_summary.empty:
    plt.figure(figsize=(10,5))
    plt.bar(exp_summary["Metric"], exp_summary["Value"])
    plt.title("Mortimer Jordan HS — Educator Experience (ALSDE 2024)")
    plt.ylabel("Percent")
    plt.tight_layout()
    add_footer(plt.gca(), "Source: ALSDE Supporting Data • 2024 • All groups (Jefferson County / MJHS)")
    out2 = CHARTS / "mjhs_educator_experience_2024.png"
    plt.savefig(out2, dpi=200); plt.show()
    print("Saved chart:", out2)

# Save derived CSVs
(DATA_PROCESSED / "mjhs_acc_kpi.csv").write_text(acc_kpi.to_csv(index=False))
(DATA_PROCESSED / "mjhs_educator_experience.csv").write_text(exp_summary.to_csv(index=False))
print("Saved processed CSVs in", DATA_PROCESSED)
